# Рекомендательная система "Подбор объектов недвижимости" 

Набор данных Bangalore House Price


- AREA_TYPE : тип района, где расположен объект недвижимости.
- AVAILABILITY : доступна ли недвижимость в настоящее время или нет.
- LOCATION : Место, где находится недвижимость.
- SIZE : Количество спален
- SOCIETY : Тип района, в котором находится собственность.
- TOTAL_SQFT : общая площадь объекта в квадратных футах.
- BATH : Количество ванных комнат
- BALCONY : Количество балконов
- PRICE : Стоимость недвижимости в лакхах¶

В качестве задания вам предлагается повторить данные действия для набора данных
- [Using chemical analysis to determine the origin of wines](https://archive.ics.uci.edu/dataset/109/wine)

In [32]:
import numpy as np
import pandas as pd

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/yakushinav/omo/main/data/Bengaluru_House_Data.csv')
print(df.head())

              area_type   availability                  location       size  \
0  Super built-up  Area         19-Dec  Electronic City Phase II      2 BHK   
1            Plot  Area  Ready To Move          Chikka Tirupathi  4 Bedroom   
2        Built-up  Area  Ready To Move               Uttarahalli      3 BHK   
3  Super built-up  Area  Ready To Move        Lingadheeranahalli      3 BHK   
4  Super built-up  Area  Ready To Move                  Kothanur      2 BHK   

   society  total_sqft  bath  balcony   price  
0  Coomee       1056.0   2.0      1.0   39.07  
1  Theanmp      2600.0   5.0      3.0  120.00  
2      NaN      1440.0   2.0      3.0   62.00  
3  Soiewre      1521.0   3.0      1.0   95.00  
4      NaN      1200.0   2.0      1.0   51.00  


In [25]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13317 entries, 0 to 13316
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13287 non-null  object 
 1   availability  13287 non-null  object 
 2   location      13286 non-null  object 
 3   size          13271 non-null  object 
 4   society       7807 non-null   object 
 5   total_sqft    13287 non-null  float64
 6   bath          13214 non-null  float64
 7   balcony       12679 non-null  float64
 8   price         13287 non-null  float64
dtypes: float64(4), object(5)
memory usage: 936.5+ KB
None


In [26]:
# Данное поле содержит очень много пропусков, удаляем
df.drop('society',axis=1,inplace=True)

In [27]:
# Заменяем пропуски в данных
df.location.replace(to_replace=np.NaN,value=df.location.mode()[0],inplace=True)
df.area_type.replace(to_replace=np.NaN,value=df.area_type.mode()[0],inplace=True)
df['size']=df['size'].str.split(' ',expand=True)[0].astype(np.number)
df['size'] = df['size'].fillna(1)
df.bath=df.bath.astype(np.number)
df['bath'] = df['bath'].fillna(1)
df.balcony = df.balcony.astype(np.number)
df['balcony'] = df['balcony'].fillna(0)
df.availability=(df.availability=='Ready To Move')
df.availability.replace({True:1,False:0},inplace=True)
df['availability'] = df['availability'].astype(np.number)

#Заменяем пропуски в данных на основе интерполяции
df['total_sqft'] = df['total_sqft'].interpolate(method='polynomial', order=2)
df['price'] = df['price'].interpolate(method='polynomial', order=2)

In [28]:
# Далее заменяем категориальные данные на числовые коды
encoder = LabelEncoder()
df['area_type'] = encoder.fit_transform(df['area_type'])
df['area_type']=df['area_type'].astype(np.number)
df['location'] = encoder.fit_transform(df['location'])
df['location'] = df['location'].astype(np.number)

In [29]:
 # Строим модель обучения по методу ближайших соседей
model = NearestNeighbors(metric='cosine')
model.fit(df)

NearestNeighbors(metric='cosine')

In [30]:
# Читаем ввод пользователя
X = {'area_type': 0,
             'availability': 1,
             'location': 300,
             'size': 2,
             'total_sqft': 300,
             'bath': 1,
             'balcony': 1,
             'price': 300}
# Создаем набор данных для построения рекомендации
x = pd.DataFrame(columns=X.keys())
x.loc[0]=X.values()

In [31]:
# Вводим количество объектов в рекомендации
k=3
# Строим рекомендацию
y = model.kneighbors(x, return_distance=False, n_neighbors=k)
# Выводим найденные объекты на экран
print('Вашему запросу соответствуют следующие объекты недвижимости:')
for x in y[0]:
    print("---------",x,"------------")
    print(df.loc[x])

Вашему запросу соответствуют следующие объекты недвижимости:
--------- 9193 ------------
area_type          3.0
availability       0.0
location        1249.0
size               3.0
total_sqft      1496.0
bath               2.0
balcony            1.0
price            718.0
Name: 9193, dtype: float64
--------- 7881 ------------
area_type          3.0
availability       1.0
location        1191.0
size               4.0
total_sqft      2000.0
bath               3.0
balcony            2.0
price           1063.0
Name: 7881, dtype: float64
--------- 7655 ------------
area_type         0.0
availability      1.0
location        964.0
size              1.0
total_sqft      425.0
bath              1.0
balcony           1.0
price           750.0
Name: 7655, dtype: float64
